# IIA 2020/2021 - Procura com Adversário
#### (12 a 16 de Outubro)

### Conteúdos
* Representação de jogos
    * classe Game
* O exemplo do jogo do galo
* Minimax (para árvore completa)
* Alfabeta (para árvore completa)
* Vamos jogar alguns jogos
* Alfabeta com profundidade limitada


### Introdução

Nesta aula vamos dar exemplos do uso dos **algoritmos minimax** e a sua variante **alfabeta/minimax para árvores completas** para o jogo do Galo que também é modelizado.

Relembrando que para caracterizar um jogo temos que identificar:

    O estado inicial do jogo;
    Quais as acções admissíveis em cada estado do jogo
        recebe: um estado do jogo
        devolve: uma lista de ações possíveis
    A função de transição
        recebe: um estado do jogo; uma jogada possível
        devolve: o estado resultante da execução dessa jogada;
    O teste de terminação
        recebe: um estado do jogo
        devolve: verdadeiro (se é o fim do jogo) ou falso
    A função de recompensa/utilidade
        recebe: um estado final do jogo
        devolve: a utilidade (para o jogador MAX)

#### Recursos necessários:
Para executar as experiências que se seguem, copie os seguintes ficheiros para a sua directoria de trabalho:
* jogos.py - módulo principal
* utils.py - módulo auxiliar

Começamos por importar o módulo principal:

In [1]:
from jogos import *

##  O Jogo do Galo

![figura](tictactoe.gif)

O estado do Jogo do Galo, vai ser modelizado:

Os jogadores são representados pelas strings 'X' (jogador MAX) e 'O' (jogador MIN).

Cada jogada é a coordenada (linha,coluna) que o jogador ocupa. Assumam que a primeira linha é a de cima e que a primeira coluna é a da esquerda.

O tabuleiro é um dicionário que guarda a informação apenas sobre as posições ocupadas.

Teremos assim que ter:

    A indicação de quem joga a seguir. Não era absolutamento necessário porque com o número de casas já jogadas poderíamos determiná-lo: um número par de casas jogadas, indica que é o 'X' a jogar e 'O' quando um número ímpar.
   
    A última casa que foi jogada. Este elemento vai ser importante para determinarmos se alguém ganhou. Para saber se alguém ganhou, basta verificar se a última jogada preencheu uma linha, coluna ou diagonal e não vale a pena testar as outras linhas, colunas e diagonais que não envolvam a casa jogada. Se não registássemos a útima jogada teríamos de testar todas as linhas, colunas e diagonais para ver se o jogo terminou, e assim poupa-se cálculos.

Vamos usar um **namedtuple** para representar o estado com os três atributos. Mas precisamos de associar alguns métodos a essa classe, por isso criamos uma sua subclasse e vamos associar-lhe esses métodos: 

    compute_utility(): o que calcula se algum dos jogadores ganhou (1 se X ganhou, -1 se O ganhou e 0 nos outros casos). Esta função faz uso de k_in_row() e other().
    
    k_in_row(): queremos saber se a partir da última jogada há K peças iguais à última em linha, coluna ou diagonal, numa certa direcção, em ambos os sentidos.
    
    O método ***k_pieces()*** verifica a existência de k símbolos iguais, seguidos, numa das quatro direcções, a partir de uma jogada de um determinado jogador (horizontal - (0,1), vertical - (1,0), e as duas diagonais - (1,1) e (1,-1).
    
    next_move(): Devolve um novo estado do jogo que resulta de uma jogada para uma determinada casa do tabuleiro.
    
    used_cells(): devolve as casas do tabuleiro onde foram jogadas as peças, até ao momento.
    
    other(): devolve o outro jogador dado player: X se O ou O se X
    
    display(): faz o display do tabuleiro que tem h linhas e v colunas.

In [2]:
stateTicTacToe = namedtuple('stateTicTacToe', 'to_move, board, last_move')

class EstadoTicTacToe(stateTicTacToe):
    
    def next_move(self,move):
        board = self.board.copy() # Sim, temos de duplicar o board.
        board[move] = self.to_move ## adiciona jogada ao dicionário (board)  
        return EstadoTicTacToe(to_move=self.other(self.to_move),
                         board=board,last_move=move)
    
    def used_cells(self):
        return self.board.keys()
    
    # devíamos dar-lhe outro nome!!!!!!!
    # confuso, confuso face a utility da classe Game
    def k_pieces(self,k):
        "If 'X' wins with this move, return 1; if 'O' wins return -1; else return 0."
        (play,board,move)=self
        if move=="None":
            return 0
        player = self.other(play) # the one thar has played, not the one that will play
        if (self.k_in_row(board, move, player, (0, 1),k) or
                self.k_in_row(board, move, player, (1, 0),k) or
                self.k_in_row(board, move, player, (1, -1),k) or
                self.k_in_row(board, move, player, (1, 1),k)):
            return 1 if player == 'X' else -1
        else:
            return 0
    
    def k_in_row(self, board, move, player,delta_x_y,k):
        "Return true if there is a line with k cells through move on board for player."
        (delta_x, delta_y) = delta_x_y
        x, y = move
        n = 0  # n is number of moves in row
        while board.get((x, y)) == player:
            n += 1
            x, y = x + delta_x, y + delta_y
        x, y = move
        while board.get((x, y)) == player:
            n += 1
            x, y = x - delta_x, y - delta_y
        n -= 1  # Because we counted move itself twice
        return n >= k
    
    def other(self,player):
        return 'X' if player == 'O' else 'O'
    
    def display(self,h,v):
        for x in range(1, h + 1):
            for y in range(1, v + 1):
                print(self.board.get((x, y), '.'), end=' ')
            print()


Criemos um estado correspondente ao tabuleiro vazio, em que inicia o jogador 'X'.

In [3]:
estado_inicial = EstadoTicTacToe(to_move = 'X', board = {}, last_move='None')

Podemos aceder aos elementos do tuplo do modo convencional

In [4]:
estado_inicial[0]

'X'

In [5]:
list(estado_inicial.used_cells())

[]

Mas para isso não criaríamos um nametuple mas um tuplo standard. Podemos também chamar os elementos pelo nome:

In [6]:
estado_inicial.last_move

'None'

Podemos imprimir o namedtuple

In [7]:
print(estado_inicial)

EstadoTicTacToe(to_move='X', board={}, last_move='None')


ou imprimir cada um dos elementos, pelo nome que demos:

In [8]:
print("Próximo jogador:",estado_inicial.to_move)
print("Tabuleiro:",estado_inicial.board)
print("Última jogada:",estado_inicial.last_move)

Próximo jogador: X
Tabuleiro: {}
Última jogada: None


Por exemplo, para o momento do jogo:

``` 
 . . X 
 . O . 
 . . .
```

o tuplo correspondente teria os seguintes valores nas suas componentes:

In [9]:
estado=EstadoTicTacToe(to_move ='X',board = {(1,3) : 'X', (2,2) : 'O'},last_move = (2,2))
print(estado)

EstadoTicTacToe(to_move='X', board={(1, 3): 'X', (2, 2): 'O'}, last_move=(2, 2))


Também podemos fazer o seu display:

In [10]:
estado.display(3,3)

. . X 
. O . 
. . . 


### A classe Game

Esta classe funciona como uma classe abstracta. Para definir um jogo concreto é necessário criar uma sua sub-classe e definir, pelo menos, os seguintes métodos:

***actions(self, state)*** : Dado um estado do jogo, este método deverá gerar todas as jogadas possíveis a partir desse estado.

***result(self, state, move)*** : Dados um estado do jogo e uma jogada válida, este método deverá retornar o estado do jogo que resulta de executar a jogada no estado dado.

***utility(self, state, player)*** : Dado um estado de jogo que seja final, e um jogador, este método deverá retornar a utilidade do estado, para o jogador. Notem que o mesmo tabuleiro terá eventualmente diferentes utilidades para os jogadores 'X' e 'O'.

***terminal_test(self, state)*** : Dado um estado do jogo, este método deverá retornar True, se o estado for final, e False, caso contrário.

In [11]:
class TicTacToe(Game):
    """Play TicTacToe on an h x v board (h is the height of the board, and v the width), 
    with Max (first player) playing 'X'. k is the number of continuous marks to win a game.
    A state has the player to move, a cached utility, a list of moves in
    the form of a list of (x, y) positions (coordinates of a move), and a board, in the form of
    a dict of {(x, y): Player} entries, where Player is 'X' or 'O'."""

    def __init__(self, h=3, v=3, k=3):
        "The board is empty, it is 'X' that begins, and no last move"
        self.h = h
        self.v = v
        self.k = k
        #moves = [(x, y) for x in range(1, h + 1)
         #        for y in range(1, v + 1)]
        self.initial = EstadoTicTacToe(to_move='X',board={},last_move="None")

    def actions(self, state):
        "Legal moves are any square not yet taken."
        return list([(x, y) for x in range(1, self.h + 1)
                 for y in range(1, self.v + 1)] - state.used_cells())

    def result(self, state, move):
        "Dado state executa jogada move"
        return state.next_move(move)
    
    
    def utility(self, state, player):
        "Return the value to player; 1 for win, -1 for loss, 0 otherwise."
        "If the player is X and .utility == 1 then return .utility"
        "Otherwise return the symmetric. Note that the symmetric of 0 is 0"
        "Note that player might be different from the player within the state that has just virtually played"
        aux = state.k_pieces(self.k)
        return aux if player == 'X' else -aux

    def terminal_test(self, state):
        "A state is terminal if someone won or there are no empty squares."
        "It assumes that the calculus if there is a winner is computed first and saved in .utility, thus it uses the value of .utility."
        return state.k_pieces(self.k) != 0 or len(self.actions(state)) == 0

    def display(self, state):
        print("Tabuleiro actual:")
        state.display(self.h,self.v)
        fim = self.terminal_test(state)
        if  fim:
            print("FIM do Jogo")
#            out = self.compute_utility(state)
#            if out == 1:
#                print('Ganhou X')
#            elif out == -1:
#                print('Ganhou O')
#            else:
#                print('Empate')
        else :
            print("Próximo jogador:{}\n".format(state.to_move))
    

Notar no código acima:

O construtor __init__(). No qual é definido o atributo initial, que representa o estado inicial do jogo. A definição deste atributo é obrigatória. Aqui é possível definir o tamanho do tabuleiro (altura x largura) e quantos simbolos continuos é necessário para ganhar o jogo. Por exemplo, podemos ter um tabuleiro de 4x6 (h=4, v=6) e o jogo termina quando um jogador conseguir fazer 3 (k=3) em linha.

Os métodos ***actions()***, ***result()*** e ***utility()***, todos de definição obrigatória.

O método ***utility()*** devolve a utilidade de um estado do ponto de vista de um dos jogadores. Se ele ganhar +1, se perder -1, de resto 0. Qualquer estado tem uma ***utility()*** e, neste caso, não se distingue um empate de uma situação em que o jogo não chegou ainda ao fim, todos serão avaliados com utilidade 0.

O método ***terminal_test()*** sucede quando o jogo acaba: se houve algum jogador que ganhou ou se não há jogadas para fazer (empate).

O método ***display()*** que mostra o estado do jogo em modo texto.

Podemos então criar um jogo e apresentá-lo...

In [12]:
j=TicTacToe()
j.display(j.initial)

Tabuleiro actual:
. . . 
. . . 
. . . 
Próximo jogador:X



Podemos ver quais as acções possíveis a partir do estado inicial do jogo:

In [13]:
j.display(j.initial)
print(j.initial.to_move,'pode jogar em',j.actions(j.initial))

Tabuleiro actual:
. . . 
. . . 
. . . 
Próximo jogador:X

X pode jogar em [(3, 2), (3, 1), (1, 1), (2, 3), (3, 3), (1, 3), (1, 2), (2, 2), (2, 1)]


Se quisermos escolher uma acção ao acaso podemos fazer o seguinte:

In [14]:
jog=random.choice(j.actions(j.initial))
print(jog)

(1, 1)


Se 'X' fizer essa jogada:

In [15]:
print('Depois de X jogar para', jog)
est1=j.result(j.initial,jog)
j.display(est1)

Depois de X jogar para (1, 1)
Tabuleiro actual:
X . . 
. . . 
. . . 
Próximo jogador:O



Podemos agora ver quais as 8 jogadas possíveis para o jogador 'O':

In [16]:
print(j.actions(est1))

[(3, 2), (3, 1), (2, 3), (3, 3), (1, 3), (1, 2), (2, 2), (2, 1)]


Imaginando que o 'O' joga para a casa (1,2):

In [17]:
est2 = j.result(est1,(1,2))
j.display(est2)

Tabuleiro actual:
X O . 
. . . 
. . . 
Próximo jogador:X



Verifiquemos se o estado resultante é terminal:

In [18]:
j.terminal_test(est2)

False

O método ***utility()***, do ponto de vista do jogador 'X' e considerando que a última jogada foi em (2,2) pelo adversário, 'O', terá que devolver 0, porque nenhum deles ganhou:

In [19]:
j.utility(est2,'X')

0

#### Exercício 1:

Teste o ***terminal_test()*** para um estado em que um dos jogadores ganhou, por exemplo
```python
O X X 
X O X 
O O X
```

Note que uma das 3 casas com X na coluna da direita terá de ser a última jogada feita e que o próximo jogador a jogar tem de ser o O. Atenção que o teste de 3 em linha é sempre feita com base na última jogada.

In [20]:
### Resolução do exercício 1
est = j.result(j.initial, (2,1))
est = j.result(est, (1,1))
est = j.result(est, (1,2))
est = j.result(est, (3,1))
est = j.result(est, (1,3))
est = j.result(est, (2,2))
est = j.result(est, (2,3))
est = j.result(est, (3,2))
j.display(est)
j.terminal_test(est)
est = j.result(est, (3,3))
j.display(est)
j.terminal_test(est)

Tabuleiro actual:
O X X 
X O X 
O O . 
Próximo jogador:X

Tabuleiro actual:
O X X 
X O X 
O O X 
FIM do Jogo


True

#### Exercício 2:
Teste o método ***utility()*** para esse estado, do ponto de vista de 'X' e do ponto de vista de 'O'.

In [21]:
# Resolução do exercício 2
print(j.utility(est,'X'))
print(j.utility(est,'O'))


1
-1


### Jogadores ao acaso

Podemos definir tipos de jogadores através de métodos que recebem um objecto do tipo ***Game*** e um objecto correspondente ao estado. Esse método deve calcular a jogada que o jogador escolhe fazer quando estiver nesse estado. No fundo é a "policy" ou estratégia.

Notem que em jogos_ia.py há um jogador que escolhe a sua jogada ao acaso.

```python
def random_player(game, state):
    """A player that chooses a legal move at random."""
    return random.choice(game.actions(state))
```


É o método ***jogar()*** que permite executar um jogo entre 2 jogadores. Esse método recebe os dois jogadores (as duas "policies") como parâmetros. Este método devolve sempre +1 se foi X que ganhou, -1 de foi O ou 0 no caso de empate.

Vamos colocar dois jogadores aleatórios a jogar um contra o outro.

In [22]:
j.jogar(random_player,random_player)

Tabuleiro actual:
. . . 
. . . 
. . . 
Próximo jogador:X

Tabuleiro actual:
. . . 
. . . 
. . X 
Próximo jogador:O

Tabuleiro actual:
. . . 
. . O 
. . X 
Próximo jogador:X

Tabuleiro actual:
X . . 
. . O 
. . X 
Próximo jogador:O

Tabuleiro actual:
X . . 
O . O 
. . X 
Próximo jogador:X

Tabuleiro actual:
X X . 
O . O 
. . X 
Próximo jogador:O

Tabuleiro actual:
X X . 
O . O 
O . X 
Próximo jogador:X

Tabuleiro actual:
X X . 
O X O 
O . X 
FIM do Jogo


1

Podem executar o código mais uma vez para verem um novo jogo. 

#### Humano a jogar
Se quisermos fazer um jogo entre uma pessoa e um jogador aleatório, teremos que usar o ***query_player***.

In [23]:
j.jogar(query_player,random_player)

Tabuleiro actual:
. . . 
. . . 
. . . 
Próximo jogador:X

available moves: [(3, 2), (3, 1), (1, 1), (2, 3), (3, 3), (1, 3), (1, 2), (2, 2), (2, 1)]

Your move? 


SyntaxError: unexpected EOF while parsing (<string>, line 0)

Podem executá-lo até ao fim. Entretanto, apresentamos o código do ***query_player*** a seguir.

``` python
def query_player(game, state):
    """Make a move by querying standard input."""
#    print("current state:")
#    game.display(state)
    print("available moves: {}".format(game.actions(state)))
    print("")
    move_string = input('Your move? ')
    try:
        move = eval(move_string)
    except NameError:
        move = move_string
    return move
```

### Jogador Minimax

Qualquer jogador minimax utiliza o método ***utility()*** da subclasse de Game.

O minimax desenrola a árvore do jogo completa para escolher a sua próxima jogada. Avalia os estados do jogo que correspondem a estados terminais, através do método ***utility***.

O método ***utility()*** só é invocado quando o jogo chega ao fim, quando satisfaz o método ***terminal_test()***, e no caso do jogo do galo devolve:
     1: no caso de vitória
     0: no caso de empate
    -1: no caso de derrota 

Notem que o minimax vai até ao fim do jogo, constrói a árvore completa que, no caso do jogo do Galo, para a primeira jogada, tem profundidade 9 e tem uma ramificação que começa em 9 e que diminui com a profundidade.

Vejemos o código do minimax_player que simplesmente invoca o método ***minimax_decision()***.

```python

def minimax_player(game, state):
    """A player that chooses the best move using minimax."""
    return minimax_decision(state,game)



def minimax_decision(state, game):
    """Given a state in a game, calculate the best move by searching
    forward all the way to the terminal states. [Figure 5.3]"""

    player = game.to_move(state)

    def max_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -infinity
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v

    def min_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = infinity
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v

    # Body of minimax_decision:
    return argmax(game.actions(state),
                  key=lambda a: min_value(game.result(state, a)))

```

Este método recebe um estado e um jogo e devolve a acção entre as acções possíveis de fazer a partir do estado, que maximiza o ganho, i.e. a acção que resulta no estado com maior ganho.
Neste caso, invoca-se o método argmax com a lista de acções possíveis nesse estado e a função que se pretende maximizar.


``` python
return argmax(game.actions(state),
                  key=lambda a: min_value(game.result(state, a)))
```

E que função é essa?
Os estados sucessores do estado raíz são nós minimizadores e por isso invoca-se a função local ***min_value()*** para todos os estados que resultam das acções, e que devolve a utilidade do ponto de vista de quem minimiza. O valor a minimizar começa muito alto (+infinity)

```python

def min_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = infinity
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v
```


Notem que essa função, por sua vez, invoca a função ***max_value()***. O valor a maximizar começa muito baixo, o mais baixo possível (-infinity).

```python

def max_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -infinity
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v
```

Qualquer das duas funções devolve a utilidade do estado quando este é terminal (fim do jogo)!

Estudem as duas funções para compreenderem o código.

Vamos então verificar o que decide o minimax_search quando lhe passamos um estado de vitória garantida. Há duas casas que garantem a vitória do X: (1,3) e (1,1). Confirmemos que o jogador minimax escolhe uma delas, na verdade a primeira devido à forma como se constrói a lista de acções.

In [24]:
vitX=EstadoTicTacToe(to_move='X', board={(1,2):'X',(2,2):'X',(3,2):'O',(2,1):'O'}, last_move=(3, 2))
j.display(vitX)
print("Joga:",minimax_decision(vitX,j))

Tabuleiro actual:
. X . 
O X . 
. O . 
Próximo jogador:X

Joga: (1, 1)


Reparem que a escolha garante a vitória e é sempre a mesma (confirmem correndo mais vezes) porque a ordem das acções é fixa.

Vamos então fazer um duelo entre o minimax e o random.

In [25]:
j.jogar(minimax_player,random_player)

Tabuleiro actual:
. . . 
. . . 
. . . 
Próximo jogador:X

Tabuleiro actual:
. . . 
. . . 
. X . 
Próximo jogador:O

Tabuleiro actual:
O . . 
. . . 
. X . 
Próximo jogador:X

Tabuleiro actual:
O . . 
. . . 
X X . 
Próximo jogador:O

Tabuleiro actual:
O O . 
. . . 
X X . 
Próximo jogador:X

Tabuleiro actual:
O O . 
. . . 
X X X 
FIM do Jogo


1

E vamos agora fazer com que seja o aleatório a abrir.

In [26]:
j.jogar(random_player,minimax_player)

Tabuleiro actual:
. . . 
. . . 
. . . 
Próximo jogador:X

Tabuleiro actual:
. . . 
. . . 
X . . 
Próximo jogador:O

Tabuleiro actual:
. . . 
. O . 
X . . 
Próximo jogador:X

Tabuleiro actual:
. . X 
. O . 
X . . 
Próximo jogador:O

Tabuleiro actual:
. . X 
. O . 
X O . 
Próximo jogador:X

Tabuleiro actual:
. . X 
. O . 
X O X 
Próximo jogador:O

Tabuleiro actual:
. . X 
. O O 
X O X 
Próximo jogador:X

Tabuleiro actual:
X . X 
. O O 
X O X 
Próximo jogador:O

Tabuleiro actual:
X O X 
. O O 
X O X 
FIM do Jogo


-1

#### Exercício 3
Façam mais do que um jogo entre dois jogadores minimax. O que acontece? Empatam sempre?
Além disso, todos os jogos se repetem, jogada após jogada. Altere o ***método minimax_decision()*** de modo a não depender da ordem fixa das acções no estados.
Volte a fazer jogos entre dois jogadores minimaxes dos novos. Os jogos não se repetem agora?

In [ ]:
# Resolva aqui o exercício 3


Faça um ou mais jogos contra o jogador minimax.

In [29]:
j.jogar(query_player,minimax_player)

Tabuleiro actual:
. . . 
. . . 
. . . 
Próximo jogador:X

available moves: [(3, 2), (3, 1), (1, 1), (2, 3), (3, 3), (1, 3), (1, 2), (2, 2), (2, 1)]

Your move? 2,2
Tabuleiro actual:
. . . 
. X . 
. . . 
Próximo jogador:O

Tabuleiro actual:
. . . 
. X . 
O . . 
Próximo jogador:X

available moves: [(3, 2), (1, 1), (2, 3), (3, 3), (1, 3), (1, 2), (2, 1)]

Your move? 1,1
Tabuleiro actual:
X . . 
. X . 
O . . 
Próximo jogador:O

Tabuleiro actual:
X . . 
. X . 
O . O 
Próximo jogador:X

available moves: [(3, 2), (2, 3), (1, 3), (1, 2), (2, 1)]

Your move? 1,2
Tabuleiro actual:
X X . 
. X . 
O . O 
Próximo jogador:O

Tabuleiro actual:
X X . 
. X . 
O O O 
FIM do Jogo


-1

#### Exercício 4
Vejam a evolução deste jogo.

``` python

Tabuleiro actual:
. . . 
. . . 
. . . 
Próximo jogador:X

available moves: [(1, 2), (3, 2), (1, 3), (3, 3), (3, 1), (2, 1), (2, 3), (2, 2), (1, 1)]

Your move? 2,1
Tabuleiro actual:
. . . 
X . . 
. . . 
Próximo jogador:O

Tabuleiro actual:
. . . 
X . . 
O . . 
Próximo jogador:X

available moves: [(1, 2), (3, 2), (1, 3), (3, 3), (2, 3), (2, 2), (1, 1)]

Your move? 1,1
Tabuleiro actual:
X . . 
X . . 
O . . 
Próximo jogador:O

Tabuleiro actual:
X . . 
X . . 
O O . 
Próximo jogador:X

available moves: [(1, 2), (1, 3), (3, 3), (2, 3), (2, 2)]

Your move? (1,3)
Tabuleiro actual:
X . X 
X . . 
O O . 
Próximo jogador:O

Tabuleiro actual:
X O X 
X . . 
O O . 
Próximo jogador:X
```

Notem que jogador O poderia ter ganho mais cedo. Porque será que o minimax escolheu assim? Na verdade, quando ele sabe que pode ganhar de várias maneiras possíveis, todas essas jogadas são as melhores para ele (valem todas 1 para ele) e não dependem de serem mais cedo ou mais tarde, valem todas 1.

Refaça a função utilidade de modo a que ganhar para um jogador seja um valor muito alto, mas que é mais alto quanto mais cedo. Replique o jogo contra o minimax cruel e se ele vencer logo que possa é bom sinal.

**Dica:**
Notem que o número de peças no tabuleiro (board) indica o quão avançado vai o jogo. Assim, uma vitória de quem está a decidir vale sempre o máximo (1) - número-peças-tabuleiro/total-casas-tabuleiro. A derrota dele ou vitória do outro vale sempre -1 + número-peças-tabuleiro/total-casas-tabuleiro. Assumimos que o adversário também queira ganhar mais rapidamente. Não são cruéis os nossos jogadores. 

In [ ]:
# Resolva o exercício 4 aqui


#### Exercício 5
Crie uma nova versão do método ***minimax_decision*** que pode chamar de ***minimax_decision_count*** que imprima no ecrã, no final, o número de estados explorados durante o processo de escolha da melhor jogada. Não use a versão shuffle, mas a standard.
Crie um jogador ***minimax_player_count()*** que usa essa nova versão do minimax e coloque-o a jogar contra o jogador random ou contra si próprio. 

In [ ]:
# resolva aqui o exercício 5


### Alfabeta

Vamos agora olhar para o método alfabeta de acordo com o algoritmo dado na teórica, considerando a versão que explora toda a árvore de jogo e que só é usada em jogos pequenos como o caso do jogo do Galo. Notem que este método não é o mesmo do aimas-python nem a que é usada no livro da disciplina do Stuart & Norvig. Esse foi renomeado para ***alphabeta_search_old()***, no ficheiro jogos.py.


``` python
def alphabeta_search(state, game):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = game.to_move(state)

    # Functions used by alphabeta
    def max_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        for a in game.actions(state):
            alpha = max(alpha, min_value(game.result(state, a), alpha, beta))
            if alpha >= beta:
                return alpha
        return alpha

    def min_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        for a in game.actions(state):
            beta = min(beta, max_value(game.result(state, a), alpha, beta))
            if beta <= alpha:
                return beta
        return beta

    # Body of alphabeta_cutoff_search:
    best_score = -infinity
    beta = infinity
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

```

Notem que o método simplesmente calcula a acção que maximiza o ganho dos sucessores, obtido através da invocação do método local ***min_value()***, passando-lhe o valor corrente de alfa (o maior ganho dos sucessores até ao momento) e o mesmo beta (+infinity).

O método ***max_value*** recebe sempre o alfa e o beta e se o estado é terminal então avalia a sua utilidade senão compara sempre o alfa com o ganho do sucessor (que é minimizador).
Sempre que o ganho é maior do que o alfa corrente torna-se o novo alfa. No final, ou porque se esgotam os sucessores ou porque há corte (alfa >= beta) o valor devolvido é o alfa.

O método ***min_value*** recebe sempre o alfa e o beta e quando o estado é terminal devolve a utilidade, senão compara sempre o beta com o ganho do sucessor (que é maximizador).
Sempre que o ganho é menor do que o beta corrente torna-se o novo beta. No final, ou porque se esgotam os sucessores ou porque há corte (alfa >= beta) o valor devolvido é o beta

O jogador alphabeta está assim definido:

```python
def alphabeta_player(game, state):
    """A player that chooses a legal move at random."""
return alphabeta_search(state,game)
```

Vamos executar um jogo entre o alfabeta e o minimax

In [ ]:
j.jogar(alphabeta_player,minimax_player)

#### Exercício 6
Desenvolva a variação do ***alphabeta_search()*** que calcule o número de estados avaliados durante o processo de procura, chamando-lhe de ***alphabeta_count_search()***.
Compare o número de estados explorados durante o processo de decisão da jogada de abertura, por exemplo, entre o ***alphabeta_count_search()*** e o ***minimax_count_search()***.

In [ ]:
#### Resolva aqui o Exercício 6


#### Exercício 7
Desenvolva o ***alphabeta_count_plus*** que é a variante alphabeta do exercício anterior em que se calcula o número de ramos cortados para além dos estados avaliados durante o processo de procura. Note que quando se chega ao fim dos sucessores com alfa >= beta não há cortes, porque não há mais filhos, não há nada para cortar.

In [ ]:
#### Resolva aqui o Exercício 7


### O Alfabeta com profundidade limitada

Vamos usar a função **alphabeta_cutoff_search_new()** que recebe um estado, um jogo, a profundidade, um teste de corte e uma função de avaliação. Podem ver os pormenores deste algoritmo no ficheiro jogos.py.

### Funções de avaliação para estados não terminais
O ficheiro ***jogos_py*** disponibilza também a função ***alphabeta_cutoff_search_new(state,game,depth,cutoff_test,eval_fn)*** que permite a realização do algoritmo alfabeta com testes de corte e utilizando funções de avaliação para os estados não terminais.
Vejamos como definir e utilizar a primeira função de avaliação:
<img src="pesos_tictactoe.PNG" width="30%">

Aqui as casas do tabuleiros têm pesos: 5 para o centro, 2 para os cantos e 1 para as restantes, refectindo a sua importância. Ao calcular a função de avaliação para um dado jogador, soma-se os valores das casas onde estão as suas peças e subtraem-se os valores das casas das peças do adversário.

Para estes 3 tabuleiros:
```python
. . . 
X O . 
. . . 
```

    Para o X vale 1-5 =-4
    Para o O vale 5-1 = 4
    
```python  
X X O 
O O X 
X O X
```
    Para o X vale 8-9 = -1
    Para o O vale 9-8 = 1

```python  
O X . 
. O . 
X X O
```
    Para o X vale 4-9 = -5
    Para o O vale 9-4 = 5

In [12]:
from rastros import *
from jogos import *
import math
    
def teste(player1, player2, n):
    score_p1 = 0
    score_p2 = 0
    i = 0
    ##########################################
    for x in range(n//2):
        jogo = jogaRastros11(player1, player2)
        if jogo[1] > 0:
            score_p1 += 1
        else:
            score_p2 += 1
        i+=1
        print("Status: ", (i/n)*100, "%")
        jogo = jogaRastros11(player2, player1)
        if jogo[1] > 0:
            score_p2 += 1
        else:
            score_p1 += 1
        i+=1
        print("Status: ", (i/n)*100, "%")
    print("----------RESULTS----------")
    print("Games processed: ", n)
    print("Score ",player1.nome,": ",score_p1)
    print("Score ",player2.nome,": ",score_p2)
    print(player1.nome," winrate: ", (score_p1/n)*100, "%")
    return (score_p1, score_p2)

def initial_graph(estado) :
    
    graph = {}
    for i in range(1,9):
        for j in range(1,9):
            graph[(i,j)] = dict.fromkeys([p for p in [(i+a, j+b) for a in [-1,0,1] for b in [-1,0,1]]
                            if p not in estado.blacks and p != (i,j) and p in estado.fullboard], 1)
    return graph  


def Dijkstra(graph,source,target,estado):
    
    # These are all the nodes which have not been visited yet
    unvisited_nodes=graph
    # It will store the shortest distance from one node to another
    shortest_distance={}
    # This will store the Shortest path between source and target node 
    route=[] 
    # It will store the predecessors of the nodes
    predecessor={}
    
    # Iterating through all the unvisited nodes
    for nodes in unvisited_nodes:
        
    # Setting the shortest_distance of all the nodes as infinty
        shortest_distance[nodes]=math.inf
        
    # The distance of a point to itself is 0.
    shortest_distance[source]=0
    
    # Running the loop while all the nodes have been visited
    while(unvisited_nodes):
        
        # setting the value of min_node as None
        min_Node=None
        
        # iterating through all the unvisited node
        for current_node in unvisited_nodes: 
            
        # For the very first time that loop runs this will be called
            if min_Node is None:
            
            # Setting the value of min_Node as the current node
                min_Node=current_node
                
            elif shortest_distance[min_Node] > shortest_distance[current_node]:
                
            # I the value of min_Node is less than that of current_node, set 
            #min_Node as current_node

                min_Node=current_node
                
        # Iterating through the connected nodes of current_node (for 
        # example, a is connected with b and c having values 10 and 3 
        # respectively) and the weight of the edges

        for child_node,value in unvisited_nodes[min_Node].items():

            # checking if the value of the current_node + value of the edge 
            # that connects this neighbor node with current_node
            # is lesser than the value that distance between current nodes 
            # and its connections
            if value + shortest_distance[min_Node] < shortest_distance[child_node]:  
                
     # If true  set the new value as the minimum distance of that connection
                shortest_distance[child_node] = value + shortest_distance[min_Node]
                
           # Adding the current node as the predecessor of the child node
                predecessor[child_node] = min_Node
        
        # After the node has been visited (also known as relaxed) remove it from unvisited node
        unvisited_nodes.pop(min_Node)
        
    # Till now the shortest distance between the source node and target node 
    # has been found. Set the current node as the target node 
    node = target
    
    # Starting from the goal node, we will go back to the source node and 
# see what path we followed to get the smallest distance
    while node != source:
        
        # As it is not necessary that the target node can be reached from # the source node, we must enclose it in a try block
        try:
            route.insert(0,node)
            node = predecessor[node]
        except Exception:
            #print('\nPath not reachable')
            #estado.display()
            break
    # Including the ssource in the path
    route.insert(0,source)
    
    # If the node has been visited,
    if shortest_distance[target] != math.inf:
        return shortest_distance[target]
    else:
        return 0
        # print the shortest distance and the path taken
        #print('Shortest distance is ' + str(shortest_distance[target]))
        #print('And the path is ' + str(route))
    # Remove the below comment if you want to show the the shortest distance 
    #from source to every other node
    # print(shortest_distance)

def fun_aval_XX(estado, jogador):
    #print(initial_graph(estado))
    if estado.terminou == 1:
        return 100 if jogador == "S" else -100
    elif estado.terminou == -1:
        return 100 if jogador == "N" else -100     
    else:
        obj = (8, 1) 
        if jogador == "N":
            obj = (1, 8)
        return 100 - Dijkstra(initial_graph(estado), estado.white, obj, estado)
    

def POSITIVA_CARALHO(estado, jogador):
    if estado.terminou == 1:
        return 10 if jogador == "S" else -10
    elif estado.terminou == -1:
        return 10 if jogador == "N" else -10     
    else:
        mul = 1
        if estado.white[0] > 4 and estado.white[1] < 4:
            mul = 4
        elif estado.white[0] < 4 and estado.white[1] > 4:
            mul = -4
        self_score = estado.white[0] + (estado.white[1] * -1)
        
        if jogador == "N":
            self_score *= -1
            if mul != 1:
                mul *= -1
        
        return mul * self_score
        
        
print(distancia((1,8),(8,1)))    
babujo = Jogador("Babujo",
                  lambda game, state:
                  alphabeta_cutoff_search_new(state,game,5,eval_fn=POSITIVA_CARALHO))  

#jogo1 = jogaRastros11(babujo, basilio)
#print(jogo1)
#mostraJogo(jogo1[0], verbose = True)

teste(babujo, basilio, 1000)
#faz_campeonato([babujo, bacoco, obtusoSW, obtusoNE, arlivre, basilio])
#dijkstra()

7
Status:  1.0 %
Status:  2.0 %
Status:  3.0 %
Status:  4.0 %
Status:  5.0 %
Status:  6.0 %
Status:  7.000000000000001 %
Status:  8.0 %
Status:  9.0 %
Status:  10.0 %
Status:  11.0 %
Status:  12.0 %
Status:  13.0 %
Status:  14.000000000000002 %
Status:  15.0 %
Status:  16.0 %
Status:  17.0 %
Status:  18.0 %
Status:  19.0 %
Status:  20.0 %
Status:  21.0 %
Status:  22.0 %
Status:  23.0 %
Status:  24.0 %
Status:  25.0 %
Status:  26.0 %
Status:  27.0 %
Status:  28.000000000000004 %
Status:  28.999999999999996 %
Status:  30.0 %
Status:  31.0 %
Status:  32.0 %
Status:  33.0 %
Status:  34.0 %
Status:  35.0 %
Status:  36.0 %
Status:  37.0 %
Status:  38.0 %
Status:  39.0 %
Status:  40.0 %
Status:  41.0 %
Status:  42.0 %
Status:  43.0 %
Status:  44.0 %
Status:  45.0 %
Status:  46.0 %
Status:  47.0 %
Status:  48.0 %
Status:  49.0 %
Status:  50.0 %
Status:  51.0 %
Status:  52.0 %
Status:  53.0 %
Status:  54.0 %
Status:  55.00000000000001 %
Status:  56.00000000000001 %
Status:  56.99999999999999 %


(3, 97)

No essencial, a função de avaliação poderá ser algo como (recorde como é representada a componente board dum estado):

In [ ]:
def f_aval_1(estado,jogador) :
    tabela = { (1,1) : 2 , (1,2) : 1, (1,3) : 2,
               (2,1) : 1 , (2,2) : 5, (2,3) : 1,
               (3,1) : 2 , (3,2) : 1, (3,3) : 2 }
    soma = 0
    for p,j in estado.board.items() :
        if j == jogador :
            soma += tabela[p]
        else :
            soma -= tabela[p]
    return soma

De modo a podermos utilizar esta função de avaliação no contexto do método jogar(), temos que definir um jogador que a use:

In [ ]:
def jogador_alfabeta_lim3f1(jogo,estado) :
    return alphabeta_cutoff_search_new(estado,jogo,3,eval_fn=f_aval_1)

Note que definimos este jogador de modo a que explore a árvore apenas até ao nível de profundidade 3; não ligámos ao teste de corte e passámos aval_1 como sendo a função de avalaição utilizada. (Ignorem o teste de corte!)

Experimentemos esta função contra o jogador aleatório:

In [ ]:
j.jogar(random_player,jogador_alfabeta_lim3f1)

Mas notem que esta função de avaliação não avalia de modo diferente a vitória, derrota ou empate. Se quisermos (o que é desejável), identificar e valorizar especialmente os estados finais, convém acrescentar algo mais:

In [ ]:
def f_aval_2(estado,jogador) :
    tabela = { (1,1) : 2 , (1,2) : 1, (1,3) : 2,
               (2,1) : 1 , (2,2) : 5, (2,3) : 1,
               (3,1) : 2 , (3,2) : 1, (3,3) : 2 }
    len_tab = 9
    if estado.k_pieces == 1 : # final e ganhou 'X'
        valor = 100 if jogador == 'X' else -100
    elif estado.k_pieces == -1 : # final e ganhou 'O'
        valor = 100 if jogador == 'O' else -100
    elif len(estado.board) == 9 : # final e empate
        valor = 0
    else:
        soma = 0
        for p,j in estado.board.items() :
            if j == jogador :
                soma += tabela[p]
            else :
                soma -= tabela[p]
        valor = soma
    return valor


Criemos o jogador alfabeta a limite 3 que usa esta última função de avaliação estática.

In [ ]:
def jogador_alfabeta_lim3f2(jogo,estado) :
    return alphabeta_cutoff_search_new(estado,jogo,3,eval_fn=f_aval_2)

Vamos pô-lo a jogar contra o aleatório.

In [ ]:
j.jogar(random_player,jogador_alfabeta_lim3f2)

E agora faremos dois jogos entre o jogador com f_val_1 e o que usa f_val_2

In [ ]:
j.jogar(jogador_alfabeta_lim3f1,jogador_alfabeta_lim3f2)

In [ ]:
j.jogar(jogador_alfabeta_lim3f2,jogador_alfabeta_lim3f1)